In [1]:
# 1. Instalación de dependencias (ejecutar en Colab)
import importlib.util, subprocess, sys
def ensure_package(package_name):
    if importlib.util.find_spec(package_name) is None:
        print(f"Instalando {package_name} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
    else:
        print(f"{package_name} ya instalado.")

ensure_package("optuna")
ensure_package("xgboost")
ensure_package("kaggle")
ensure_package("joblib")

print("Dependencias listas.")



Instalando optuna ...
xgboost ya instalado.
kaggle ya instalado.
joblib ya instalado.
Dependencias listas.


In [2]:
# 2. Imports
import os, zipfile
import numpy as np
import pandas as pd
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from joblib import dump

print("Imports cargados.")



Imports cargados.


In [3]:

# 3. Configurar Kaggle y descargar datos

from google.colab import files

from google.colab import files
print("Sube tu kaggle.json (API token de Kaggle) cuando se abra el diálogo...")
uploaded = files.upload()

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
for fname in uploaded.keys():
    if fname == "kaggle.json":
        dest = os.path.join(os.path.expanduser("~/.kaggle"), "kaggle.json")
        os.replace(fname, dest)
        os.chmod(dest, 0o600)
        print("kaggle.json copiado a ~/.kaggle")
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -p /content
!unzip -o /content/udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip -d /content/udea_saber_pro

print("Descarga y extracción completadas.")


Sube tu kaggle.json (API token de Kaggle) cuando se abra el diálogo...


Saving kaggle.json to kaggle.json
kaggle.json copiado a ~/.kaggle
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.50GB/s]
Archive:  /content/udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: /content/udea_saber_pro/submission_example.csv  
  inflating: /content/udea_saber_pro/test.csv  
  inflating: /content/udea_saber_pro/train.csv  
Descarga y extracción completadas.


In [4]:

# 4. Cargar train y test

TRAIN_PATH = "/content/udea_saber_pro/train.csv"
TEST_PATH  = "/content/udea_saber_pro/test.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)

print("Train shape:", train.shape)
print("Test shape:", test.shape)

display(train.head())
display(test.head())



Train shape: (692500, 21)
Test shape: (296786, 20)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [5]:
# 5. Función reusable de imputación (media / moda)
def imputar_media_moda(df, id_col=None, target_col=None):
    """
    Imputa media en numéricas y moda en categóricas (ignora id y target)
    Retorna: df_imputado, columnas_numericas, columnas_categoricas
    """
    df_imputado = df.copy()
    excluir = set()
    if id_col and id_col in df_imputado.columns:
        excluir.add(id_col)
    if target_col and target_col in df_imputado.columns:
        excluir.add(target_col)
    columnas_numericas = df_imputado.select_dtypes(include=[np.number]).columns.tolist()
    columnas_categoricas = df_imputado.select_dtypes(include=['object','category','bool']).columns.tolist()
    columnas_numericas = [c for c in columnas_numericas if c not in excluir]
    columnas_categoricas = [c for c in columnas_categoricas if c not in excluir]
    for col in columnas_numericas:
        if df_imputado[col].isna().any():
            df_imputado[col] = df_imputado[col].fillna(df_imputado[col].mean())
    for col in columnas_categoricas:
        if df_imputado[col].isna().any():
            moda = df_imputado[col].mode(dropna=True)
            if len(moda) > 0:
                df_imputado[col] = df_imputado[col].fillna(moda.iloc[0])
    return df_imputado, columnas_numericas, columnas_categoricas

print("Función de imputación definida.")


Función de imputación definida.


In [6]:
# 6. Definir preprocesador con ColumnTransformer
ID_COL = "ID"
TARGET_COL = "RENDIMIENTO_GLOBAL"

train_imp, num_cols, cat_cols = imputar_media_moda(train, id_col=ID_COL, target_col=TARGET_COL)
feature_cols = [c for c in train.columns if c not in [ID_COL, TARGET_COL]]
numeric_features = [c for c in feature_cols if c in num_cols]
categorical_features = [c for c in feature_cols if c in cat_cols]

numeric_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="mean"))])
categorical_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="most_frequent")),
                                          ("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

print("Preprocesador definido.")
print("num features:", len(numeric_features), "cat features:", len(categorical_features))



Preprocesador definido.
num features: 5 cat features: 14


In [7]:
# 7. Definir X, y, aplicar label mapping y split
X = train[feature_cols].copy()
y = train[TARGET_COL].copy()

# Mapeo obligatorio: texto -> números (XGBoost necesita labels numéricos en multiclass)
label_map = {
    "bajo": 0,
    "medio-bajo": 1,
    "medio-alto": 2,
    "alto": 3
}

# Si y es texto/categoría mapeamos; si ya es numérico, no lo tocamos
if y.dtype == object or y.dtype.name == "category":
    y = y.map(label_map)

num_clases = len(set(y.unique()))
print("num_clases:", num_clases)

# Train/valid split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
print("X_train:", X_train.shape, "X_valid:", X_valid.shape)


num_clases: 4
X_train: (484750, 19) X_valid: (207750, 19)


In [8]:
# 8. Función para construir pipelines
def make_base_pipeline(model):
    return Pipeline(steps=[("preprocess", preprocessor), ("model", model)])

print("make_base_pipeline listo.")



make_base_pipeline listo.


In [9]:
# 9. Submuestreo para Optuna (ajusta train_size si quieres más/menos)
X_optuna, _, y_optuna, _ = train_test_split(X_train, y_train, train_size=0.3, stratify=y_train, random_state=42)
print("Subconjunto para Optuna:", X_optuna.shape)



Subconjunto para Optuna: (145425, 19)


In [10]:
# 10. Objetivos de Optuna para XGB, SVM, KNN, RF

import warnings
from sklearn.exceptions import FitFailedWarning


def objective_knn(trial):

    # Submuestreo adicional SOLO para KNN
    X_sub, _, y_sub, _ = train_test_split(
        X_optuna, y_optuna,
        train_size=0.3,
        stratify=y_optuna,
        random_state=trial.number
    )

    n_neighbors = trial.suggest_int("n_neighbors", 3, 10, step=2)
    p = trial.suggest_categorical("p", [1, 2])
    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights='distance', p=p)
    pipe = make_base_pipeline(model)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(pipe, X_optuna, y_optuna, cv=cv, scoring='accuracy', n_jobs=-1, error_score='raise')
    return scores.mean()


def objective_rf(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 100)
    max_depth = trial.suggest_int("max_depth", 3, 20)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5)
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   class_weight='balanced', random_state=42, n_jobs=-1)
    pipe = make_base_pipeline(model)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(pipe, X_optuna, y_optuna, cv=cv, scoring='accuracy', n_jobs=-1, error_score='raise')
    return scores.mean()

print("Objetivos Optuna definidos.")


Objetivos Optuna definidos.


In [11]:
# 11. Ejecutar Optuna (ajusta n_trials según tiempo disponible)
n_trials_knn = 3
n_trials_rf  = 5

study_knn = optuna.create_study(direction='maximize', study_name='knn_saberpro')
study_knn.optimize(objective_knn, n_trials=n_trials_knn)
print("Best KNN:", study_knn.best_params)

study_rf = optuna.create_study(direction='maximize', study_name='rf_saberpro')
study_rf.optimize(objective_rf, n_trials=n_trials_rf)
print("Best RF:", study_rf.best_params)


[I 2025-11-26 19:36:19,432] A new study created in memory with name: knn_saberpro
/usr/local/lib/python3.12/dist-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [3, 10] and step=2, but the range is not divisible by `step`. It will be replaced with [3, 9].
  warnings.warn(
[I 2025-11-26 19:45:55,707] Trial 0 finished with value: 0.3612239986247207 and parameters: {'n_neighbors': 9, 'p': 1}. Best is trial 0 with value: 0.3612239986247207.
/usr/local/lib/python3.12/dist-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [3, 10] and step=2, but the range is not divisible by `step`. It will be replaced with [3, 9].
  warnings.warn(
[I 2025-11-26 19:55:27,469] Trial 1 finished with value: 0.3567612171222279 and parameters: {'n_neighbors': 7, 'p': 1}. Best is trial 0 with value: 0.3612239986247207.
/usr/local/lib/python3.12/dist-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [3, 10] and s

Best KNN: {'n_neighbors': 9, 'p': 1}


[I 2025-11-26 20:05:00,821] Trial 0 finished with value: 0.38780814853017015 and parameters: {'n_estimators': 61, 'max_depth': 9, 'min_samples_split': 2, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.38780814853017015.
[I 2025-11-26 20:05:15,866] Trial 1 finished with value: 0.4007632800412584 and parameters: {'n_estimators': 97, 'max_depth': 18, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.4007632800412584.
[I 2025-11-26 20:05:22,526] Trial 2 finished with value: 0.39308234485129795 and parameters: {'n_estimators': 90, 'max_depth': 12, 'min_samples_split': 7, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.4007632800412584.
[I 2025-11-26 20:05:30,190] Trial 3 finished with value: 0.396101083032491 and parameters: {'n_estimators': 90, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.4007632800412584.
[I 2025-11-26 20:05:35,198] Trial 4 finished with value: 0.3900704830668729 and parameters: {'n

Best RF: {'n_estimators': 97, 'max_depth': 18, 'min_samples_split': 5, 'min_samples_leaf': 2}


In [12]:
# 12. Definir pipelines finales con parámetros tuneados

best_knn = KNeighborsClassifier(**study_knn.best_params, weights='distance')
knn_pipeline = make_base_pipeline(best_knn)

best_rf = RandomForestClassifier(**study_rf.best_params, class_weight='balanced', random_state=42, n_jobs=-1)
rf_pipeline = make_base_pipeline(best_rf)

print('Pipelines finales listos.')


Pipelines finales listos.


In [13]:
# 13. Entrenar, evaluar y guardar resultados
modelos_finales = {'KNN': knn_pipeline, 'RandomForest': rf_pipeline}

resultados = []
mejores_modelos = {}

for nombre, pipe in modelos_finales.items():
    print('\\n==============================')
    print(f'🚀 Entrenando modelo: {nombre}')
    print('==============================')
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_valid)
    acc = accuracy_score(y_valid, y_pred)
    print(f'Accuracy {nombre}: {acc:.4f}')
    print('\\nClassification report:')
    print(classification_report(y_valid, y_pred))
    resultados.append({'Modelo': nombre, 'Accuracy': acc})
    mejores_modelos[nombre] = pipe
    dump(pipe, f'modelo_entrenado_{nombre}.joblib')

df_resultados = pd.DataFrame(resultados).sort_values('Accuracy', ascending=False)
display(df_resultados)


\n==============================
🚀 Entrenando modelo: KNN
Accuracy KNN: 0.3672
\nClassification report:
              precision    recall  f1-score   support

           0       0.39      0.42      0.41     51896
           1       0.29      0.28      0.29     51682
           2       0.28      0.25      0.27     51486
           3       0.48      0.51      0.50     52686

    accuracy                           0.37    207750
   macro avg       0.36      0.37      0.36    207750
weighted avg       0.36      0.37      0.36    207750

\n==============================
🚀 Entrenando modelo: RandomForest
Accuracy RandomForest: 0.4016
\nClassification report:
              precision    recall  f1-score   support

           0       0.41      0.56      0.47     51896
           1       0.32      0.22      0.26     51682
           2       0.30      0.22      0.25     51486
           3       0.49      0.61      0.54     52686

    accuracy                           0.40    207750
   macro avg 

,Modelo,Accuracy
1,RandomForest,0.401627
0,KNN,0.367187
